## Importing data

In [1]:
# import necessary modules for data analysis and visualisation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
patient = pd.read_csv('./data/sim_av_patient.csv', delimiter=',')
sact_patient = pd.read_csv('./data/sim_sact_patient.csv', delimiter=',')

In [3]:
patient.head()

,PATIENTID,SEX,LINKNUMBER,ETHNICITY,DEATHCAUSECODE_1A,DEATHCAUSECODE_1B,DEATHCAUSECODE_1C,DEATHCAUSECODE_2,DEATHCAUSECODE_UNDERLYING,DEATHLOCATIONCODE,NEWVITALSTATUS,VITALSTATUSDATE
0,10000001,2,810000001,A,NaN,NaN,NaN,NaN,NaN,NaN,A,2017-01-17
1,10000002,2,810000002,Z,NaN,NaN,NaN,NaN,NaN,NaN,A,2017-01-14
2,10000003,1,810000003,A,NaN,NaN,NaN,NaN,NaN,NaN,A,2017-01-17
3,10000004,1,810000004,A,NaN,NaN,NaN,NaN,NaN,NaN,A,2017-01-13
4,10000005,2,810000005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,2017-01-16


In [4]:
patient['DEATHLOCATIONCODE'].value_counts()

1    138327
4     98681
2     40284
X     28209
5     17025
6      7146
3       709
Name: DEATHLOCATIONCODE, dtype: int64

## About this table
This table contains 12 columns, which are about the synthetic patients' demographic information and vital status, as well as death cause of those who are deceased. The columns are:
- Patient ID
- Sex
- Link Number (allows linkeage to another table)
- Ethnicity
- Death Cause Code * 5 (1A, 1B, 1C, 2, Underlying)
- Death Location Code
- Vital Status
- Vital Status Date

There are 1,322,100 entries in this table.

In [5]:
patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1322100 entries, 0 to 1322099
Data columns (total 12 columns):
PATIENTID                    1322100 non-null int64
SEX                          1322100 non-null int64
LINKNUMBER                   1322100 non-null int64
ETHNICITY                    1192249 non-null object
DEATHCAUSECODE_1A            330280 non-null object
DEATHCAUSECODE_1B            98085 non-null object
DEATHCAUSECODE_1C            18106 non-null object
DEATHCAUSECODE_2             141982 non-null object
DEATHCAUSECODE_UNDERLYING    327910 non-null object
DEATHLOCATIONCODE            330381 non-null object
NEWVITALSTATUS               1322100 non-null object
VITALSTATUSDATE              1322100 non-null object
dtypes: int64(3), object(9)
memory usage: 121.0+ MB


## Data Cleaning

### Inconsistent delimiters in death cause code columns
There are several columns in this table which represent death causes of the patients. These death causes are recorded with ICD-10 codes and if a patient died because multiple factors, their codes are separated by commas.

However, in some of the rows, the delimiter is space(' '), semicolon(';'), or full-stop('.') rather than comma, which should be fixed.

In [6]:
# Define a function which is used to turn all other delimiters in death cause code columns with commas.
def to_comma(code):
    if type(code) == str:
        if code.count('.') > 0 and any(x.isdigit() for x in code): # Replace full-stops with commas.
            code = code.replace('.', ',')
            return code
        elif code.count(';') > 0 and any(x.isdigit() for x in code):  # Replace semicolons with commas.
            code = code.replace(';', ',')
            return code
        elif code.count(' ') > 0 and any(x.isdigit() for x in code):   # Replace space with commas.
            code = code.replace(' ', ',')
            return code
      
        else:
            return code
    else:
        return code
    
    
# Apply the function to all the death cause code columns with `Series.map()`.
patient['DEATHCAUSECODE_1A'] = patient['DEATHCAUSECODE_1A'].map(to_comma)
patient['DEATHCAUSECODE_1B'] = patient['DEATHCAUSECODE_1B'].map(to_comma)
patient['DEATHCAUSECODE_1C'] = patient['DEATHCAUSECODE_1C'].map(to_comma)
patient['DEATHCAUSECODE_2'] = patient['DEATHCAUSECODE_2'].map(to_comma)
patient['DEATHCAUSECODE_UNDERLYING'] = patient['DEATHCAUSECODE_UNDERLYING'].map(to_comma)

In [7]:
# patient[patient['PATIENTID'] == 140025403]
patient[patient['PATIENTID'] == 30059690]

,PATIENTID,SEX,LINKNUMBER,ETHNICITY,DEATHCAUSECODE_1A,DEATHCAUSECODE_1B,DEATHCAUSECODE_1C,DEATHCAUSECODE_2,DEATHCAUSECODE_UNDERLYING,DEATHLOCATIONCODE,NEWVITALSTATUS,VITALSTATUSDATE
515368,30059690,1,830059690,A,K922,"BILATEARL PULMONARY EMBOLISM, ASPIRATION PNEUM...",NaN,J459,J449,1,D,2016-06-17


After converting all the inconsistent delimiters, I tried to split all the cells containing multiple death causes and calculate how many times a death cause code showed in this table, this is to see:
1. If all the delimiters were correctly turned into commas.
2. If there was any other value that was not recoreded in ICD-10 codes.

In [8]:
death_causes = {} # Init a dict to store all the death cause codes and how many times.

# Define a function to split multiple codes and calculate the numbers of each code.
def calculate_death_causes(code):
    if type(code) == str:
        for i in code.split(','):
            if i.strip() not in death_causes and i != '':
                death_causes[i.strip()] = 1  # If the death code is not in the dict, put it in and set its number to 1.
            elif i != '':
                death_causes[i.strip()] += 1 # Otherwise, add 1 to its number.
    else:
        pass

# Apply this function to all the death cause code columns.
patient['DEATHCAUSECODE_1A'].map(calculate_death_causes)
patient['DEATHCAUSECODE_1B'].map(calculate_death_causes)
patient['DEATHCAUSECODE_1C'].map(calculate_death_causes)
patient['DEATHCAUSECODE_2'].map(calculate_death_causes)
patient['DEATHCAUSECODE_UNDERLYING'].map(calculate_death_causes)

# Turn the dict into a Pandas Series Object.
death_causes_series = pd.Series(death_causes)
death_causes_series['ASPIRATION PNEUMONIA']

1

Length of all the indicdes in the new Series should be less than 5 characters, according to ICD-10 Code standard. However, I then found some descriptive phrases as below about patiens' death causes but they were not recorded in ICD-10 codes.

There were many problems with these phrases. For example:
- **Inconsistent**: same causes but in different ways of recording: 'ISCHAEMIC HEART DISEASE' and 'Ischaemic Heart Disease'.
- **Words lost**: like 'ASPIRATION', which can be 'ASPIRATION PNEUMONIA'.
- **Ambiguous**: like 'CANCER OF LUNG' and 'PNEUMONIA'.
- **Not listed in ICD-10**: like 'CEREBROVASCULAR ACCIDENT'

In [54]:
dc_not_coded = death_causes_series[np.where(death_causes_series.index.map(len) >= 5)[0]]
dc_not_coded

ISCHAEMIC HEART DISEASE            1
BRONCHOPNEUMONIA                   1
CEREBROVASCULAR ACCIDENT           2
DEEP VEIN THROMBOSIS               3
ACUTE CARDIAC FAILURE              1
RESPIRATORY FAILURE                2
CARDIORESPIRATORY FAILURE          2
PNEUMONIA                          1
CARCINOMATOSIS                     1
CORONARY ATHEROSCLEROSIS           1
ISCHAEMIC STROKE                   1
BILATEARL PULMONARY EMBOLISM       1
ASPIRATION PNEUMONIA               1
COMPLETELY DISABLED                3
CANCER OF LUNG                     2
ISCHAEMIC HEART DISEASE.           2
CORONARY ARTERY ATHEROSCLEROSIS    2
HODGKINS LYMPHOMA                  2
ACUTE KIDNEY INJURY                1
CHRONIC KIDNEY DISEASE             2
CONGESTIVE CARDIAC FAILURE         2
HYPERTENSION                       2
Ischaemic                          1
Heart                              1
Disease                            1
DEMENTIA                           1
ASPIRATION                         1
d

In [63]:
# This is to check if any entry in a death cause code column contains death causes not listed in the ICD-10 code.
# It returns a list which cosist of True(for not listed) and False.

def check_dc_not_coded(values):
    arr = []
    for value in values:
        if type(value) == str:
            if any(i.strip() in dc_not_coded.index for i in value.split(',')):
                print(patient[patient['DEATHCAUSECODE_1A'] == value].index)
                arr.append(True)
            else:
                arr.append(False)
        else:
            arr.append(False)
    return arr

# Collect all the rows where the death causes are not listed.

dc_to_drop = np.where(np.array(check_dc_not_coded(patient['DEATHCAUSECODE_1A']))
+ np.array(check_dc_not_coded(patient['DEATHCAUSECODE_1B']))
+ np.array(check_dc_not_coded(patient['DEATHCAUSECODE_1C']))
+ np.array(check_dc_not_coded(patient['DEATHCAUSECODE_2']))
+ np.array(check_dc_not_coded(patient['DEATHCAUSECODE_UNDERLYING'])))[0]


# Drop those rows in place;
patient.drop(dc_to_drop, inplace=True)
patient.shape

# dc_to_drop = np.concatenate((np.where(check_dc_not_coded(np.array(patient['DEATHCAUSECODE_1A'])))[0], 
#             np.where(check_dc_not_coded(np.array(patient['DEATHCAUSECODE_1B'])))[0],
#             np.where(check_dc_not_coded(np.array(patient['DEATHCAUSECODE_1C'])))[0],
#             np.where(check_dc_not_coded(np.array(patient['DEATHCAUSECODE_2'])))[0],
#             np.where(check_dc_not_coded(np.array(patient['DEATHCAUSECODE_UNDERLYING'])))[0]))

# patient.drop(dc_to_drop)
# dc_to_drop

#     np.where(check_dc_not_coded(patient['DEATHCAUSECODE_1A'])
#          + check_dc_not_coded(patient['DEATHCAUSECODE_1B'])
#          + check_dc_not_coded(patient['DEATHCAUSECODE_1C'])
#          + check_dc_not_coded(patient['DEATHCAUSECODE_2'])
#          + check_dc_not_coded(patient['DEATHCAUSECODE_UNDERLYING']))[0]

# def check_dc_not_coded(values):
#     if type(values) == str:
#         for i in values.split(','):
#             if i.strip() in dc_not_coded.index:
#                 print(i)
#                 return True
#         else:
#             return False
#     else:
#         return False

# check_dc_not_coded(patient['DEATHCAUSECODE_1A'].values)
# for i in patient["DEATHCAUSECODE_1A"].values:
#     if type(i) == str:
#         if len(i) > 5:
#             if i.count(',') == 0:
#                 print(i)


#     
# len(np.where(np.array([check_dc_not_coded(x) for x in patient['DEATHCAUSECODE_1B'].values]))[0])
        
# len(np.concatenate((np.where(np.array([check_dc_not_coded(x) for x in patient['DEATHCAUSECODE_1A'].values]))[0],
#                np.where(np.array([check_dc_not_coded(x) for x in patient['DEATHCAUSECODE_1B'].values]))[0],
#                np.where(np.array([check_dc_not_coded(x) for x in patient['DEATHCAUSECODE_1C'].values]))[0],
#                np.where(np.array([check_dc_not_coded(x) for x in patient['DEATHCAUSECODE_2'].values]))[0],
#                np.where(np.array([check_dc_not_coded(x) for x in patient['DEATHCAUSECODE_UNDERLYING'].values]))[0])))


# np.where(patient['DEATHCAUSECODE_1A'].values.apply("split(',')").isin(dc_not_coded.index))[0]

# patient.drop(np.where(patient[['DEATHCAUSECODE_1A', 'DEATHCAUSECODE_1B', 'DEATHCAUSECODE_1C', 'DEATHCAUSECODE_2', 'DEATHCAUSECODE_UNDERLYING']].isin(dc_not_coded.index))[0])
# np.where(patient['DEATHCAUSECODE_2'].valstr.contains('Ischaemic'))
# patient['DEATHCAUSECODE_2'].values
# np.flatnonzero(np.core.defchararray.find(patient['DEATHCAUSECODE_2'].values, 'Ischaemic')!=-1)
# patient.drop(np.where(patient['DEATHCAUSECODE_1A'].isin(dc_not_coded.index))[0])

# # type(x)
# # t = patient['DEATHCAUSECODE_1A'].isin(x)

# # t = np.array([True,True,True,False])
# # np.where(t)[0]
# # [i for i, x in enumerate(t) if x]

(1322066, 12)

### Date format conversion and errors on vital status date
First we turned the format of vital status dates into `Pandas Timestamp`, which can be easily compared and calculated. Then we found 31 outliers where the date was on 01/01/1990. These can be blank values in database and we decided to drop these entries.

In [65]:
# Turn the vital status data into Pandas Timestamp 
patient['VITALSTATUSDATE'] = pd.to_datetime(patient['VITALSTATUSDATE'])

# Drop the rows with an error on date by comparing with another Timestamp
patient.drop(patient[patient['VITALSTATUSDATE'] < pd.Timestamp('2000-01-01 00:00:00')].index, inplace=True)
patient.shape

# This line is to check if these entries are dropped and should return no entries
# patient[patient['VITALSTATUSDATE'] < pd.Timestamp('2000-01-01 00:00:00')]

(1322035, 12)

### Inconsistent column name
This table can be linked to the Systemic Anti-Cancer Therapy (SACT) tables (sim_sact) by an overlapped column 'LINKNUMBER' but the names were different in the two tables, so we changged it to 'LINK_NUMBER' in order to keep consistent.

In [66]:
# Change the column name to 'LINK_NUMBER' in place.
patient.rename(columns={'LINKNUMBER': 'LINK_NUMBER'}, inplace=True)

# This line is to check if the name is correctly changed and should return a Series
# patient['LINK_NUMBER']

## Demographic Information

[Ethnicity Category Code](https://www.datadictionary.nhs.uk/data_dictionary/attributes/e/end/ethnic_category_code_de.asp)

In [ ]:
# Group the data by sex and ethnicity to see the demographic information of patients.
group_a = patient.groupby(['SEX', 'ETHNICITY' ]).count()

# The proportion of ethnic groups among male patients
m_ethnicity = group_a['PATIENTID'][0:20]
m_ethnicity_codes = [x[1] for x in m_ethnicity.index]
ethnicity_names = ['', '', 'British', 'Irish', 'Any other White background', 'White and Black Caribbean', 'White and Black African', 'White and Asian', 'Any other mixed background', 'Indian', 'Pakistani', 'Bangladeshi', 'Any other Asian background', 'Caribbean', 'African', 'Any other black background', 'Chinese', 'Any other ethnic group', '', 'Not stated']
m_labels = list(zip(m_ethnicity_codes, ethnicity_names))

# # The proportion of ethnic groups among non-British male patients
m_non_british = m_ethnicity.drop(labels=(1, 'A'))
m_non_british_labels = list(zip(m_ethnicity_codes, ethnicity_names))
m_non_british_labels.remove(('A', 'British'))

In [ ]:
# Visualise the proportion in two pie charts
fig1, ax1 = plt.subplots(1, 2, figsize=(24, 24))
ax1[0].pie(m_ethnicity, textprops={'fontsize': 8})
ax1[1].pie(m_non_british, labels= [x[0] for x in m_non_british_labels])
ax1[0].set_title('Proportion of ethnic groups(Male)')
ax1[1].set_title('Proportion of ethnic groups(Male, exclude British)')
ax1[0].legend(m_labels)
ax1[1].legend(m_non_british_labels)

plt.show()

In [ ]:
# The proportion of ethnic groups among female patients
fm_ethnicity = group_a['PATIENTID'][20:40]
fm_ethnicity_codes = [x[1] for x in fm_ethnicity.index]
fm_labels = list(zip(fm_ethnicity_codes, ethnicity_names))

# The proportion of ethnic groups among non-british female patients
fm_non_british = fm_ethnicity.drop(labels=(2, 'A'))
fm_non_british_labels = list(zip(fm_ethnicity_codes, ethnicity_names))
fm_non_british_labels.remove(('A', 'British'))

In [ ]:
# Visualise the proportion in two pie charts
fig2, ax2 = plt.subplots(1, 2, figsize=(24, 24))
ax2[0].pie(fm_ethnicity)
ax2[1].pie(fm_non_british, labels=[x[0] for x in fm_non_british_labels])
ax2[0].set_title('Proportion of ethnic groups(Female)')
ax2[1].set_title('Proportion of ethnic groups(Female, exclude British)')
ax2[0].legend(fm_labels)
ax2[1].legend(fm_non_british_labels)

plt.show()

In [ ]:
group_b = patient.groupby('DEATHCAUSECODE_1A').count()
# group_b
# group_b['PATIENTID'].plot()

In [ ]:
patient['DEATHLOCATIONCODE'].value_counts().sum()

## Death Location

In [ ]:
patient['DEATHLOCATIONCODE'].value_counts()

## Death Causes

142,407 patients died because of bronchus or lung 

In [ ]:
death_causes_1a = pd.Series(death_cause_1a_num)

print(death_causes_1a.idxmax(), death_causes_1a.max())
print(death_causes_1a.idxmin(), death_causes_1a.min())

## Reference
[Ethnicity Category Code](https://www.datadictionary.nhs.uk/data_dictionary/attributes/e/end/ethnic_category_code_de.asp)

In [ ]:
# patient.describe()
# patient['DEATHCAUSECODE_1A'][]



In [ ]:
# patient['VITALSTATUSDATE'] = pd.to_datetime(patient['VITALSTATUSDATE'])

# patient[patient['VITALSTATUSDATE']<pd.Timestamp('2000-01-01 00:00:00')]

# clean_data = patient.drop(patient[patient['VITALSTATUSDATE'] == '1900-01-01'].index)
# sns.scatterplot(x=clean_data['VITALSTATUSDATE'], y=clean_data['VITALSTATUSDATE'])
# patient['VITALSTATUSDATE'][0]
# plt.show()